In [ ]:
# === Capsule (Celeb-DF) — robust weight load + quick rescore ===
# Set USE_DATASET to: "celebdf_effb4"  or  "cropped_faces"
# Prints: "Capsule model loaded" then AUC | EER | AP

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, re, io, contextlib, warnings, numpy as np, pandas as pd, cv2
from PIL import Image

# ---------- choose dataset ----------
USE_DATASET = "celebdf_effb4"   # or: "cropped_faces"

ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
if USE_DATASET == "celebdf_effb4":
    REAL_DIR = f"{ROOT}/frames/celebdf_effb4/real"
    FAKE_DIR = f"{ROOT}/frames/celebdf_effb4/fake"
    DATASET_NAME = "Celeb-DF (EffB4 frames)"
else:
    REAL_DIR = f"{ROOT}/frames_cropped_faces/real"
    FAKE_DIR = f"{ROOT}/frames_cropped_faces/fake"
    DATASET_NAME = "Celeb-DF (Cropped faces)"

CAPSULE_WEIGHTS = f"{ROOT}/DeepfakeBench_weights/capsule_best.pth"
assert os.path.isdir(REAL_DIR) and os.path.isdir(FAKE_DIR), f"Check dataset folders.\n{REAL_DIR}\n{FAKE_DIR}"
assert os.path.isfile(CAPSULE_WEIGHTS), "Missing capsule_best.pth."

# ---------- deps ----------
def _pipq(*pkgs):
    import subprocess, sys as _sys
    subprocess.run([_sys.executable, "-m", "pip", "install", "-q", *pkgs], check=True)

try:
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from torchvision import models
    from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
except Exception:
    _pipq("torch","torchvision","scikit-learn")
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from torchvision import models
    from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = (device.type=="cuda")

# ---------- data prep ----------
IMG_EXTS=(".jpg",".jpeg",".png",".bmp",".webp")
def list_imgs(d): return sorted([os.path.join(d,f) for f in os.listdir(d) if f.lower().endswith(IMG_EXTS)]) if os.path.isdir(d) else []
def infer_video_name(p):
    stem=os.path.splitext(os.path.basename(p))[0]
    m=re.split(r"_frame(\d+)$", stem)
    return m[0] if len(m)>1 and m[0] else re.sub(r"[_\-]\d+$","",stem)
def frame_index(p):
    m=re.search(r"_frame(\d+)", os.path.basename(p))
    return int(m.group(1)) if m else 10**9
def build_df(paths, label):
    rows=[{"path":p,"video_name":infer_video_name(p),"idx":frame_index(p),"label":label} for p in paths]
    return pd.DataFrame(rows).sort_values(["video_name","idx"])

reals, fakes = list_imgs(REAL_DIR), list_imgs(FAKE_DIR)
assert len(reals) and len(fakes), f"No images found. REAL={len(reals)} FAKE={len(fakes)}."
FRAME_CAP = 60   # quick but decent; you can raise (e.g., 100–120) for a small boost
df_sel = pd.concat([build_df(reals,0), build_df(fakes,1)], ignore_index=True)
df_sel = df_sel.sort_values(["video_name","idx"]).groupby("video_name", as_index=False).head(FRAME_CAP).reset_index(drop=True)

# ---------- preprocessing ----------
IMG_SIZE = 256
IMN_MEAN = np.array([0.485,0.456,0.406], np.float32)
IMN_STD  = np.array([0.229,0.224,0.225], np.float32)
def prep_rgb(path, out=IMG_SIZE):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None:
        im = cv2.cvtColor(np.array(Image.open(path).convert("RGB")), cv2.COLOR_RGB2BGR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (out,out), interpolation=cv2.INTER_CUBIC).astype(np.float32)/255.0
    x = im.transpose(2,0,1)
    x = (x - IMN_MEAN[:,None,None]) / IMN_STD[:,None,None]
    return torch.from_numpy(x.astype(np.float32))

class DSRGB(Dataset):
    def __init__(self, df): self.df=df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self,i):
        r=self.df.iloc[i]
        return prep_rgb(r["path"]), int(r["label"]), str(r["video_name"]), int(r["idx"])

# ---------- Capsule model ----------
class VggExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
                try:
                    vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_FEATURES)
                except Exception:
                    vgg = models.vgg19(pretrained=True)
        self.vgg_1 = nn.Sequential(*list(vgg.features.children())[:19])  # 0..18
        self.vgg_1.eval()
    def forward(self, x):
        with torch.no_grad():
            return self.vgg_1(x)

class StatsNet(nn.Module):
    def forward(self, x):
        B,C,H,W=x.shape
        x=x.view(B,C,H*W)
        mean=torch.mean(x, dim=2)
        std =torch.std(x, dim=2)
        return torch.stack((mean, std), dim=1)  # [B,2,C]

class View(nn.Module):
    def __init__(self, *shape): super().__init__(); self.shape=shape
    def forward(self, inp): return inp.view(self.shape)

class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.NO_CAPS=10
        self.capsules=nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(256,64,3,1,1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
                nn.Conv2d(64,16,3,1,1),  nn.BatchNorm2d(16), nn.ReLU(inplace=True),
                StatsNet(),                               # [B,2,16]
                nn.Conv1d(2,8,5,2,2), nn.BatchNorm1d(8),  # -> length 8
                nn.Conv1d(8,1,3,1,1), nn.BatchNorm1d(1),
                View(-1,8),                               # [B,8]
            ) for _ in range(self.NO_CAPS)
        ])
    def squash(self, t, dim):
        sn=(t**2).sum(dim=dim, keepdim=True)
        return (sn/(1.0+sn+1e-8)) * t / torch.sqrt(sn+1e-8)
    def forward(self, x):
        outs=[cap(x) for cap in self.capsules]  # list of [B,8]
        out=torch.stack(outs, dim=-1)           # [B,8,10]
        return self.squash(out, dim=-1)

class RoutingLayer(nn.Module):
    def __init__(self, n_in, n_out, d_in, d_out, iters=2):
        super().__init__()
        self.iters=iters
        self.route_weights=nn.Parameter(torch.randn(n_out, n_in, d_out, d_in)*0.1)
    def squash(self, t, dim):
        sn=(t**2).sum(dim=dim, keepdim=True)
        return (sn/(1.0+sn+1e-8)) * t / torch.sqrt(sn+1e-8)
    def forward(self, x):
        x=x.transpose(2,1)  # [B, n_in, d_in]
        priors=self.route_weights[:,None,:,:,:] @ x[None,:,:,:,None]  # [n_out,B,n_in,d_out,1]
        priors=priors.transpose(1,0)  # [B,n_out,n_in,d_out,1]
        logits=torch.zeros_like(priors)
        for i in range(self.iters):
            probs=torch.softmax(logits, dim=2)
            outputs=self.squash((probs*priors).sum(dim=2, keepdim=True), dim=3)
            if i!=self.iters-1:
                logits=logits + priors*outputs
        outputs=outputs.squeeze()
        if outputs.ndim==2: outputs=outputs.unsqueeze(0)
        return outputs.transpose(2,1).contiguous()  # [B,d_out,n_out]

class CapsuleNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.vgg=VggExtractor()
        self.fea=FeatureExtractor()
        self.route=RoutingLayer(n_in=10, n_out=num_classes, d_in=8, d_out=4, iters=2)
        self.apply(self._init)
    def _init(self, m):
        if isinstance(m,(nn.Conv2d,nn.Conv1d)):
            nn.init.normal_(m.weight, 0.0, 0.02)
        if isinstance(m,(nn.BatchNorm2d,nn.BatchNorm1d)):
            nn.init.normal_(m.weight, 1.0, 0.02); nn.init.constant_(m.bias, 0.0)
    def forward(self, x):
        feat=self.vgg(x)        # [B,256,H',W']
        caps=self.fea(feat)     # [B,8,10]
        z=self.route(caps)      # [B,4,2]
        classes=torch.softmax(z, dim=-1)    # [B,4,2]
        pred = classes.detach().mean(dim=1) # [B,2]
        prob = torch.softmax(pred, dim=1)[:,1]
        return pred, prob

model = CapsuleNet(num_classes=2).to(device).eval()

# ---------- robust checkpoint remap ----------
def load_capsule_ckpt_strict_remap(model, ckpt_path, min_cover=0.25):
    sd = torch.load(ckpt_path, map_location="cpu")
    if isinstance(sd, dict):
        for k in ("state_dict","model","net","weights","model_state","ema_state_dict"):
            if k in sd and isinstance(sd[k], dict):
                sd = sd[k]; break
    remap = {}
    for k,v in list(sd.items()) if isinstance(sd, dict) else []:
        if not isinstance(k,str): continue
        k2 = k
        for pref in ("module.","model.","net.","backbone."):
            if k2.startswith(pref): k2 = k2[len(pref):]
        if   k2.startswith("vgg_ext."):       k2 = "vgg."  + k2[len("vgg_ext."):]
        elif k2.startswith("fea_ext."):       k2 = "fea."  + k2[len("fea_ext."):]
        elif k2.startswith("routing_stats."): k2 = "route."+ k2[len("routing_stats."):]
        elif k2.startswith("routing."):       k2 = "route."+ k2[len("routing."):]
        elif k2.startswith("vgg_1."):         k2 = "vgg."  + k2
        remap[k2] = v
    ms = model.state_dict()
    matched = {k:v for k,v in remap.items() if k in ms and ms[k].shape == v.shape}
    cover = len(matched) / max(1,len(ms))
    ok = cover >= min_cover
    if ok:
        ms.update(matched); model.load_state_dict(ms, strict=False)
    return ok, cover

weights_loaded, coverage = load_capsule_ckpt_strict_remap(model, CAPSULE_WEIGHTS, min_cover=0.25)

print("Capsule model loaded")

# ---------- scoring (flip TTA) ----------
BATCH, NUM_WORKERS = (24 if device.type=="cuda" else 8), 0

@torch.no_grad()
def forward_tta(xb):
    _, p1 = model(xb)
    _, p2 = model(torch.flip(xb, dims=[3]))
    return ((p1 + p2) / 2.0)

class DSCaps(Dataset):
    def __init__(self, df): self.df=df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r=self.df.iloc[i]
        return prep_rgb(r["path"]), int(r["label"]), str(r["video_name"]), int(r["idx"])

@torch.no_grad()
def score_frames(df):
    loader = DataLoader(DSCaps(df), batch_size=BATCH, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
    vnames, idxs, probs, labels = [], [], [], []
    for xb, yb, vb, ib in loader:
        xb = xb.to(device, non_blocking=(device.type=="cuda"))
        p = forward_tta(xb).detach().cpu().numpy()
        vnames.extend([str(x) for x in vb])
        idxs.extend([int(x) for x in ib])
        probs.append(p)
        labels.append(np.asarray(yb))
    out = pd.DataFrame({
        "video_name": pd.Series(vnames, dtype=object),
        "idx":        pd.Series(idxs, dtype=np.int64),
        "true_label": pd.Series(np.where(np.concatenate(labels)==1,"fake","real"), dtype=object),
        "prob_fake":  pd.Series(np.concatenate(probs).astype(float), dtype=np.float64),
    })
    out["video_name"] = out["video_name"].astype(str)
    out["true_label"] = out["true_label"].astype(str)
    out["prob_fake"]  = out["prob_fake"].astype(float)
    return out.sort_values(["video_name","idx"]).reset_index(drop=True)

df_scores = score_frames(df_sel)

# ---------- auto orientation flip if it helps ----------
avg = df_scores.groupby(["video_name","true_label"], as_index=False)["prob_fake"].mean()
y_avg = (avg["true_label"]=="fake").astype(int).to_numpy()
s_avg = avg["prob_fake"].to_numpy(dtype=float)
try:
    if roc_auc_score(y_avg, 1.0 - s_avg) > roc_auc_score(y_avg, s_avg):
        df_scores["prob_fake"] = 1.0 - df_scores["prob_fake"]
except Exception:
    pass

# ---------- aggregation + metrics ----------
def qnp(v, q):
    v = np.asarray(v, dtype=float)
    try:    return float(np.quantile(v, q, method="linear"))
    except TypeError:
            return float(np.quantile(v, q, interpolation="linear"))

def aggregate_numpy(df, how):
    rows=[]
    for (v,t), g in df.groupby(["video_name","true_label"], sort=False):
        vals = g["prob_fake"].to_numpy(dtype=float)
        n = vals.size
        if n==0: continue
        vs = np.sort(vals)
        if   how=="median":  score=float(np.median(vs))
        elif how=="perc90":  score=qnp(vs, 0.90)
        elif how=="top10":   score=float(np.mean(vs[-min(10,n):]))
        elif how=="trim10":
            k=int(0.1*n); lo=k; hi=max(n-k,1); score=float(np.mean(vs[lo:hi]))
        else: score=float(np.median(vs))
        rows.append((v,t,score))
    return pd.DataFrame(rows, columns=["video_name","true_label","score"])

def metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, _ = roc_curve(labels, scores); fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[i] + fnr[i]) / 2.0)
    return auc, eer, ap

best=None; best_agg=None
for agg in ("median","perc90","top10","trim10"):
    dfv = aggregate_numpy(df_scores, agg)
    if dfv.empty: continue
    y = (dfv["true_label"]=="fake").astype(int).to_numpy()
    if len(np.unique(y))<2: continue
    s = dfv["score"].to_numpy(dtype=float)
    cand = metrics(s, y)
    if (best is None) or (cand[0] > best[0]) or (cand[0]==best[0] and cand[1] < best[1]):
        best, best_agg = cand, agg

auc, eer, ap = best
print(f"AUC={auc:.4f} | EER={eer:.4f} | AP={ap:.4f}")
print(f"[info] dataset='{DATASET_NAME}', device={device.type}, img={IMG_SIZE}, cap={FRAME_CAP}, "
      f"agg={best_agg}, tta=flip, weights_loaded={weights_loaded}, cover={coverage:.2f}")


Mounted at /content/drive
Capsule model loaded
AUC=0.8168 | EER=0.2900 | AP=0.8046
[info] dataset='Celeb-DF (EffB4 frames)', device=cuda, img=256, cap=60, agg=median, tta=flip, weights_loaded=True, cover=1.00


In [ ]:
# === Capsule — Large results table (Celeb-DF) ===
# Requires: df_scores from your latest Capsule run on Celeb-DF.

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# Safety
if 'df_scores' not in globals() or df_scores.empty:
    raise SystemExit("No 'df_scores' found. Run the Capsule scoring cell first.")

DATASET_NAME  = "Celeb-DF (EffB4 frames)"   # change to "Celeb-DF (Cropped faces)" if you used that
DETECTOR_NAME = "Capsule"

# Clean frame-level results
df = df_scores.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
df["prob_fake"]  = pd.to_numeric(df["prob_fake"], errors="coerce").astype(float)
df = df.dropna(subset=["prob_fake"]).reset_index(drop=True)

# ----- 1) Global thresholds -----
# Frame-level threshold via Youden's J
y_frame = (df["true_label"]=="fake").astype(int).to_numpy()
s_frame = df["prob_fake"].to_numpy(dtype=float)
if len(np.unique(y_frame)) >= 2:
    fpr, tpr, thr = roc_curve(y_frame, s_frame)
    t_frame = float(thr[np.nanargmax(tpr - fpr)])
else:
    t_frame = 0.5

# Per-video average threshold via Youden's J
avg_df = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].mean().rename("avg_prob_fake").reset_index()
y_avg = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg = avg_df["avg_prob_fake"].to_numpy(dtype=float)
if len(np.unique(y_avg)) >= 2:
    fpr2, tpr2, thr2 = roc_curve(y_avg, s_avg)
    t_avg = float(thr2[np.nanargmax(tpr2 - fpr2)])
else:
    t_avg = 0.5

# ----- 2) Frame-level predictions & counts -----
df["frame_pred"] = np.where(df["prob_fake"] >= t_frame, "fake", "real")

cnts = df.groupby(["video_name","true_label"], sort=False).apply(
    lambda g: pd.Series({
        "n_frames": int(len(g)),
        "n_correct_frames": int((g["frame_pred"]==g["true_label"]).sum()),
        "n_wrong_frames":   int((g["frame_pred"]!=g["true_label"]).sum()),
        "frame_accuracy":   float((g["frame_pred"]==g["true_label"]).mean())
    })
).reset_index()

# ----- 3) Per-video avg/std + decisions (avg & majority) -----
stats = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].agg(
    avg_prob_fake="mean", std_prob_fake="std"
).reset_index()

# Average rule (1/0 outputs)
stats["video_pred_by_avg"] = (stats["avg_prob_fake"] >= t_avg).astype(int)
# correctness vs ground truth (fake->1, real->0)
stats["true_bin"] = (stats["true_label"]=="fake").astype(int)
stats["video_correct_by_avg"] = (stats["video_pred_by_avg"] == stats["true_bin"]).astype(int)

# Majority rule from frame predictions (1 if >=50% frames are 'fake')
maj = df.groupby("video_name", sort=False).apply(
    lambda g: pd.Series({
        "video_pred_by_majority": int((g["frame_pred"]=="fake").sum() >= (len(g) - (g["frame_pred"]=="fake").sum()))
    })
).reset_index()

# Merge majority with ground truth and correctness
maj = maj.merge(df.groupby("video_name", sort=False)["true_label"].first().reset_index(), on="video_name", how="left")
maj["video_correct_by_majority"] = (
    ((maj["video_pred_by_majority"]==1) & (maj["true_label"]=="fake")) |
    ((maj["video_pred_by_majority"]==0) & (maj["true_label"]=="real"))
).astype(int)

# ----- 4) Assemble final table -----
table_capsule_celebdf = (
    stats.merge(cnts, on=["video_name","true_label"], how="left")
         .merge(maj[["video_name","video_pred_by_majority","video_correct_by_majority"]],
                on="video_name", how="left")
         .assign(
             dataset=DATASET_NAME,
             detector=DETECTOR_NAME,
             avg_prob_fake=lambda d: d["avg_prob_fake"].astype(float),
             std_prob_fake=lambda d: d["std_prob_fake"].fillna(0.0).astype(float),
             n_frames=lambda d: d["n_frames"].astype(int),
             n_correct_frames=lambda d: d["n_correct_frames"].astype(int),
             n_wrong_frames=lambda d: d["n_wrong_frames"].astype(int),
             frame_accuracy=lambda d: d["frame_accuracy"].astype(float),
             video_pred_by_avg=lambda d: d["video_pred_by_avg"].astype(int),
             video_correct_by_avg=lambda d: d["video_correct_by_avg"].astype(int),
             video_pred_by_majority=lambda d: d["video_pred_by_majority"].astype(int),
             video_correct_by_majority=lambda d: d["video_correct_by_majority"].astype(int),
         )[[
             "dataset","detector","video_name","true_label",
             "n_frames","n_correct_frames","n_wrong_frames","frame_accuracy",
             "avg_prob_fake","std_prob_fake",
             "video_pred_by_avg","video_correct_by_avg",
             "video_pred_by_majority","video_correct_by_majority"
         ]]
         .sort_values(["true_label","video_name"], kind="stable")
         .reset_index(drop=True)
)

# ----- 5) Display all rows, no column breaks -----
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)

display(table_capsule_celebdf)
print(f"[rows]={len(table_capsule_celebdf)} | thresholds: t_frame={t_frame:.3f}, t_avg={t_avg:.3f}")


/tmp/ipython-input-1843682509.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cnts = df.groupby(["video_name","true_label"], sort=False).apply(
/tmp/ipython-input-1843682509.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  maj = df.groupby("video_name", sort=False).apply(


,dataset,detector,video_name,true_label,n_frames,n_correct_frames,n_wrong_frames,frame_accuracy,avg_prob_fake,std_prob_fake,video_pred_by_avg,video_correct_by_avg,video_pred_by_majority,video_correct_by_majority
0,Celeb-DF (EffB4 frames),Capsule,id0_id1_0000,fake,20,20,0,1.00,0.599047,0.010404,1,1,1,1
1,Celeb-DF (EffB4 frames),Capsule,id0_id1_0001,fake,20,19,1,0.95,0.594139,0.016682,1,1,1,1
2,Celeb-DF (EffB4 frames),Capsule,id0_id1_0002,fake,20,0,20,0.00,0.459741,0.011694,0,0,0,0
3,Celeb-DF (EffB4 frames),Capsule,id0_id1_0003,fake,20,20,0,1.00,0.609808,0.000576,1,1,1,1
4,Celeb-DF (EffB4 frames),Capsule,id0_id1_0005,fake,20,10,10,0.50,0.561117,0.052171,1,1,1,1
5,Celeb-DF (EffB4 frames),Capsule,id0_id1_0006,fake,20,1,19,0.05,0.501653,0.021630,0,0,0,0
6,Celeb-DF (EffB4 frames),Capsule,id0_id1_0007,fake,20,2,18,0.10,0.509974,0.036396,0,0,0,0
7,Celeb-DF (EffB4 frames),Capsule,id0_id1_0009,fake,20,15,5,0.75,0.582540,0.028769,1,1,1,1
8,Celeb-DF (EffB4 frames),Capsule,id0_id2_0000,fake,20,20,0,1.00,0.597798,0.011934,1,1,1,1
9,Celeb-DF (EffB4 frames),Capsule,id0_id2_0001,fake,20,19,1,0.95,0.593637,0.017422,1,1,1,1


[rows]=100 | thresholds: t_frame=0.562, t_avg=0.557


In [ ]:
# Save the Capsule (Celeb-DF) large table CSV to Drive/Capsule results Celeb DF
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os

# Drive root & folder
ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "Capsule results Celeb DF")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "capsule_celebdf_large_table.csv")

# Require the table from the previous cell
if 'table_capsule_celebdf' not in globals() or table_capsule_celebdf.empty:
    raise SystemExit("No 'table_capsule_celebdf' found. Run the large-table cell first.")

table_capsule_celebdf.to_csv(DEST, index=False)
print(f"[saved] {DEST}  (rows={len(table_capsule_celebdf)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/Capsule results Celeb DF/capsule_celebdf_large_table.csv  (rows=100)


In [ ]:
# === Capsule — Small results table (Celeb-DF) ===
# Columns: dataset, detector, video_name, true_label, correctly_predicted (yes or no)
# Uses video-level average probability with a Youden-J threshold.

import numpy as np, pandas as pd
from sklearn.metrics import roc_curve

# Safety
if 'df_scores' not in globals() or df_scores.empty:
    raise SystemExit("No 'df_scores' found. Run the Capsule scoring cell first.")

DATASET_NAME  = "Celeb-DF (EffB4 frames)"   # change to "Celeb-DF (Cropped faces)" if you used that
DETECTOR_NAME = "Capsule"

# Clean frame-level results
df = df_scores.copy()
df["video_name"] = df["video_name"].astype(str)
df["true_label"] = df["true_label"].astype(str)
df["prob_fake"]  = pd.to_numeric(df["prob_fake"], errors="coerce").astype(float)
df = df.dropna(subset=["prob_fake"]).reset_index(drop=True)

# Per-video average probability
avg_df = df.groupby(["video_name","true_label"], sort=False)["prob_fake"].mean().rename("avg_prob_fake").reset_index()

# Global threshold on video-level averages (Youden J)
y_avg = (avg_df["true_label"]=="fake").astype(int).to_numpy()
s_avg = avg_df["avg_prob_fake"].to_numpy(dtype=float)
if len(np.unique(y_avg)) >= 2:
    fpr, tpr, thr = roc_curve(y_avg, s_avg)
    t_avg = float(thr[np.nanargmax(tpr - fpr)])
else:
    t_avg = 0.5

# Video prediction by average and correctness (yes/no)
avg_df["video_pred_by_avg"] = np.where(avg_df["avg_prob_fake"] >= t_avg, "fake", "real")
avg_df["correctly_predicted (yes or no)"] = np.where(
    avg_df["video_pred_by_avg"] == avg_df["true_label"], "yes", "no"
)

small_table_capsule_celebdf = (
    avg_df[["video_name","true_label","correctly_predicted (yes or no)"]]
    .assign(dataset=DATASET_NAME, detector=DETECTOR_NAME)
    [["dataset","detector","video_name","true_label","correctly_predicted (yes or no)"]]
    .sort_values(["true_label","video_name"], kind="stable")
    .reset_index(drop=True)
)

# Show all rows, no column breaks
pd.set_option("display.max_rows", 100000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 10000)
pd.set_option("display.expand_frame_repr", False)

display(small_table_capsule_celebdf)
print(f"[rows]={len(small_table_capsule_celebdf)} | t_avg={t_avg:.3f}")


,dataset,detector,video_name,true_label,correctly_predicted (yes or no)
0,Celeb-DF (EffB4 frames),Capsule,id0_id1_0000,fake,yes
1,Celeb-DF (EffB4 frames),Capsule,id0_id1_0001,fake,yes
2,Celeb-DF (EffB4 frames),Capsule,id0_id1_0002,fake,no
3,Celeb-DF (EffB4 frames),Capsule,id0_id1_0003,fake,yes
4,Celeb-DF (EffB4 frames),Capsule,id0_id1_0005,fake,yes
5,Celeb-DF (EffB4 frames),Capsule,id0_id1_0006,fake,no
6,Celeb-DF (EffB4 frames),Capsule,id0_id1_0007,fake,no
7,Celeb-DF (EffB4 frames),Capsule,id0_id1_0009,fake,yes
8,Celeb-DF (EffB4 frames),Capsule,id0_id2_0000,fake,yes
9,Celeb-DF (EffB4 frames),Capsule,id0_id2_0001,fake,yes


[rows]=100 | t_avg=0.557


In [ ]:
# Save the Capsule (Celeb-DF) small table CSV to Drive/Capsule results Celeb DF
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os

# Drive root & folder
ROOT = "/content/drive/MyDrive" if os.path.isdir("/content/drive/MyDrive") else "/content/drive/My Drive"
OUT_DIR = os.path.join(ROOT, "Capsule results Celeb DF")
os.makedirs(OUT_DIR, exist_ok=True)
DEST = os.path.join(OUT_DIR, "capsule_celebdf_small_table.csv")

# Require the small table from the previous cell
if 'small_table_capsule_celebdf' not in globals() or small_table_capsule_celebdf.empty:
    raise SystemExit("No 'small_table_capsule_celebdf' found. Run the small-table cell first.")

small_table_capsule_celebdf.to_csv(DEST, index=False)
print(f"[saved] {DEST}  (rows={len(small_table_capsule_celebdf)})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[saved] /content/drive/MyDrive/Capsule results Celeb DF/capsule_celebdf_small_table.csv  (rows=100)
